

# **Implementing-Multi-Class-Classification-and-Named-Entity-Recognition-in-Biomedical-Texts**


**Notebook Structure:**

The notebook is structured as follows:

For two datasets, **PubMed 20k RCT** and **BC5CDR** (BioCreative V CDR corpus), different NLP tasks are applied: "Multi_class Classification" for PubMed 20k RCT and "Named Entity Recognition" (NER) for BC5CDR. The models are trained on two different pre_trained BERT models: **bert-base-cased** and **biobert-base-cased-v1.2** (a pre-trained biomedical language representation model specifically designed for biomedical text mining; more information can be found here:https://academic.oup.com/bioinformatics/article/36/4/1234/5566506 ).

**Differences Between the Two Datasets:**

    PubMed 20k RCT:
        A text classification dataset.
        Each abstract is split into sentences, and each sentence is labeled with a specific section type.
        Classes: BACKGROUND, OBJECTIVE, METHODS, RESULTS, or CONCLUSIONS.

    BC5CDR:
        A Named Entity Recognition (NER) dataset.
        Token-level annotations in BIO format are used to tag diseases and chemicals.
        In BIO format:
            B- indicates the beginning of an entity (e.g., B-Disease).
            I- indicates the inside of an entity (e.g., I-Disease).
            O indicates a token that is outside any entity.


**For each dataset, the notebook is divided into the following four sections:**

1.   Setting up the GPU Environment
2.   Getting Data
3.   Training and Testing the Model
4.   Using the Model (Running Inference)

**Technical Details_Fine-tuning Parameters:**

    num_train_epochs=1: Number of epochs (this can be increased to 3 for better results; however, due to GPU limitations and to reduce training time, 1 epoch was selected).
    train_batch_size=16: Batch size for training.
    eval_batch_size=16: Batch size for evaluation.

**General Results:**

For both tasks (multi-class classification and NER), models trained on biobert-base-cased-v1.2 demonstrated better performance compared to bert-base-cased:

*PubMed 20k RCT + Task: Multi-Class Classification*

    Model: bert-base-cased
        MCC: 0.8247
        Precision: 0.8777, Recall: 0.8694, F1: 0.8711
        
    Model: biobert-base-cased-v1.2
        MCC: 0.8290
        Precision: 0.8779, Recall: 0.8726, F1: 0.8736


*BC5CDR + Task: NER*

    Model: bert-base-cased
        Precision: 0.7883, Recall: 0.8569, F1: 0.8211


    Model: biobert-base-cased-v1.2
        Precision: 0.8398, Recall: 0.8939, F1: 0.8660




# Data: PubMed 20k RCT

•	Task: Multi_class classification

•	Paper: Franck Dernoncourt, Ji Young Lee. PubMed 200k RCT: a Dataset for Sequential Sentence Classification in Medical Abstracts. International Joint Conference on Natural Language Processing (IJCNLP). 2017.

•	Details: 20,000 abstracts of randomized controlled trials.

•	Labels(targets): Methods, Results, Conclusions, Background, Objective.

•	Data source: https://github.com/Franck-Dernoncourt/pubmed-rct/tree/master



**Section 1: Setting up the GPU Environment**

In [ ]:
# Install required libraries
!pip install transformers datasets torch scikit-learn --quiet
!pip install simpletransformers --quiet
import torch
import pandas as pd
import os
import logging
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Set up logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Check for GPU

if torch.cuda.is_available():
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("Using CPU")



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

**Section 2: Getting Data**



In [ ]:
# Clone the dataset
!git clone https://github.com/Franck-Dernoncourt/pubmed-rct.git
!ls pubmed-rct


Cloning into 'pubmed-rct'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 39 (delta 8), reused 5 (delta 5), pack-reused 25 (from 1)
Receiving objects: 100% (39/39), 177.08 MiB | 22.07 MiB/s, done.
Resolving deltas: 100% (15/15), done.
Updating files: 100% (13/13), done.
PubMed_200k_RCT				       PubMed_20k_RCT_numbers_replaced_with_at_sign
PubMed_200k_RCT_numbers_replaced_with_at_sign  README.md
PubMed_20k_RCT


The downloaded repository contains four folders, each with a version of the PubMed 200k RCT dataset. According to the README:



*   PubMed 20k is a subset of PubMed 200k.
*   The _numbers_replaced_with_at_sign versions replace all numbers with @.

   

We'll focus on PubMed_20k_RCT , a smaller subset ideal for quicker experiments.

In [ ]:
data_dir= "pubmed-rct/PubMed_20k_RCT/"

# Check all of the filenames in the target directory
filenames = [data_dir + filename for filename in os.listdir(data_dir)]
filenames


['pubmed-rct/PubMed_20k_RCT/train.txt',
 'pubmed-rct/PubMed_20k_RCT/test.txt',
 'pubmed-rct/PubMed_20k_RCT/dev.txt']

In [ ]:
# Example of raw data

def get_lines(filepath):
    with open(filepath, "r") as file:
        return file.readlines()
train_lines = get_lines(data_dir+"train.txt")
train_lines[:3]

['###24293578\n',
 'OBJECTIVE\tTo investigate the efficacy of 6 weeks of daily low-dose oral prednisolone in improving pain , mobility , and systemic low-grade inflammation in the short term and whether the effect would be sustained at 12 weeks in older adults with moderate to severe knee osteoarthritis ( OA ) .\n',
 'METHODS\tA total of 125 patients with primary knee OA were randomized 1:1 ; 63 received 7.5 mg/day of prednisolone and 62 received placebo for 6 weeks .\n']

In [ ]:
# Data preprocessing

def preprocess_text(filename):
    """Returns a list of dictionaries of abstract line data.

    Takes in a filename, reads its contents, and processes each line,
    extracting the target label and the text of the sentence.

    Args:
        filename: A string of the target text file to read and extract line data from.

    Returns:
        A list of dictionaries, each containing a line's label and text.
        Example:
        [{"target": 'CONCLUSION',
          "text": "the study couldn't have gone better, turns out people are kinder than you think"}]
    """


    input_lines = get_lines(filename)  # Read all lines from the file
    abstract_lines = ""  # Create an empty string for the abstract
    abstract_samples = []  # List to hold processed abstract samples

    # Loop through each line in the file
    for line in input_lines:
        if line.startswith("###"):  # Check if the line is an ID line
            abstract_lines = ""  # Reset the abstract string
        elif line.isspace():  # Check if the line is a new line
            abstract_line_split = abstract_lines.splitlines()  # Split the abstract into separate lines

            # Process each line in the abstract
            for abstract_line in abstract_line_split:
                line_data = {}  # Dictionary to store line data
                target_text_split = abstract_line.split("\t")  # Split label and text
                line_data["target"] = target_text_split[0]  # Extract the target label
                # line_data["text"] = target_text_split[1].lower()  # Extract the sentence text
                line_data["text"] = target_text_split[1]  # Extract the sentence text
                abstract_samples.append(line_data)  # Append to the list of abstract samples
        else:  # Otherwise, it's part of an abstract
            abstract_lines += line

    return abstract_samples

train_samples = preprocess_text("pubmed-rct/PubMed_20k_RCT/train.txt")
test_samples = preprocess_text("pubmed-rct/PubMed_20k_RCT/test.txt")
dev_samples = preprocess_text("pubmed-rct/PubMed_20k_RCT/dev.txt")

# Convert to pandas DataFrame for easier manipulation

train_df = pd.DataFrame(train_samples)
test_df = pd.DataFrame(test_samples)
dev_df = pd.DataFrame(dev_samples)

print("Example of preprocessd_train_data:\n",train_df.head())


Example of preprocessd_train_data:
       target                                               text
0  OBJECTIVE  To investigate the efficacy of 6 weeks of dail...
1    METHODS  A total of 125 patients with primary knee OA w...
2    METHODS  Outcome measures included pain reduction and i...
3    METHODS  Pain was assessed using the visual analog pain...
4    METHODS  Secondary outcome measures included the Wester...


In [ ]:
#Check Dataset Sizes:
print("Training dataset size:", len(train_df))
print("Test dataset size:", len(test_df))
print("Dev dataset size:", len(dev_df))


Training dataset size: 180040
Test dataset size: 30135
Dev dataset size: 30212


In [ ]:
# Ensure the columns are in the format SimpleTransformers expects
train_df = train_df.rename(columns={"text": "text", "target": "labels"})
test_df = test_df.rename(columns={"text": "text", "target": "labels"})
dev_df = dev_df.rename(columns={"text": "text", "target": "labels"})

# Optional: Encode labels to integers
label_encoder = LabelEncoder()
train_df["labels"] = label_encoder.fit_transform(train_df["labels"])
test_df["labels"] = label_encoder.transform(test_df["labels"])
dev_df["labels"] = label_encoder.transform(dev_df["labels"])
print(train_df.head())


   labels                                               text
0       3  To investigate the efficacy of 6 weeks of dail...
1       2  A total of 125 patients with primary knee OA w...
2       2  Outcome measures included pain reduction and i...
3       2  Pain was assessed using the visual analog pain...
4       2  Secondary outcome measures included the Wester...


**Section 3: Training and Testing the Model**

In [ ]:
# Optional model configuration
model_args = ClassificationArgs(
    num_train_epochs=1,             # Number of epochs
    train_batch_size=16,            # Batch size for training
    eval_batch_size=16,             # Batch size for evaluation
    overwrite_output_dir=True,      # Overwrite the output directory
    output_dir="./ClassificationModel_bert",  # Output directory
)


In [ ]:
# Create a ClassificationModel: Pre-trained BERT model="bert-base-cased"
model_bert = ClassificationModel(
    "bert",                         # Use the BERT architecture
    "bert-base-cased",            # Pre-trained BERT model
    num_labels=len(label_encoder.classes_),  # Number of labels
    use_cuda=torch.cuda.is_available(),  # True: Use GPU if available
    args=model_args                 # Model arguments
)

# Train the model
model_bert.train_model(train_df,eval_data=dev_df)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:905: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


  0%|          | 0/60 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1884 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


Evaluation Results on fine-tuned 'bert-base-cased':  {'mcc': 0.8247442491140681, 'eval_loss': 0.36842352955457763}


In [ ]:
# Define a custom metrics function
def compute_metrics(preds, labels):
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy = accuracy_score(labels, preds)
    return {
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "accuracy": accuracy,
    }


# Evaluate the model
result, model_outputs, wrong_predictions = model_bert.eval_model(test_df, custom_metrics=compute_metrics)

# Print evaluation results
print("Evaluation Results on fine-tuned 'bert-base-cased':")
for metric, value in result.items():
    if isinstance(value, dict):
        print(f"{metric}:")
        for sub_metric, sub_value in value.items():
            print(f"  {sub_metric}: {sub_value:.4f}")
    elif isinstance(value, (int, float)):
        print(f"{metric}: {value:.4f}")
    else:
        print(f"{metric}: {value}")


  0%|          | 0/60 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1884 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


Evaluation Results on fine-tuned 'bert-base-cased':
mcc: 0.8247
custom_metrics:
  precision: 0.8777
  recall: 0.8694
  f1: 0.8711
  accuracy: 0.8694
eval_loss: 0.3684


In [ ]:
# Create a ClassificationModel: Pre-trained BERT model="biobert-base-cased-v1.2"
model_biobert = ClassificationModel(
    "bert",                         # Use the BERT architecture
    "dmis-lab/biobert-base-cased-v1.2",            # Pre-trained BERT model
    num_labels=len(label_encoder.classes_),  # Number of labels
    use_cuda=torch.cuda.is_available(),  # True: Use GPU if available
    args=model_args                 # Model arguments
)

# Train the model
model_biobert.train_model(train_df,eval_data=dev_df)
# Evaluate the model
result, model_outputs, wrong_predictions = model_biobert.eval_model(test_df, custom_metrics=compute_metrics)

# Print evaluation results
print("Evaluation Results on fine-tuned 'biobert-base-cased-v1.2':")
for metric, value in result.items():
    if isinstance(value, dict):
        print(f"{metric}:")
        for sub_metric, sub_value in value.items():
            print(f"  {sub_metric}: {sub_value:.4f}")
    elif isinstance(value, (int, float)):
        print(f"{metric}: {value:.4f}")
    else:
        print(f"{metric}: {value}")

pytorch_model.bin:  53%|#####2    | 231M/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

  0%|          | 0/360 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

...

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:905: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


  0%|          | 0/60 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1884 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


Evaluation Results on fine-tuned 'biobert-base-cased-v1.2':
mcc: 0.8290
custom_metrics:
  precision: 0.8779
  recall: 0.8726
  f1: 0.8736
  accuracy: 0.8726
eval_loss: 0.3564


**Section 4: Using the Model**

In [ ]:
# Make predictions
sample_texts = [" We prolonged the exposure period and investigated how quickly diesel exhaust can induce respiratory and systemic effects ."]
#correct lable is:BACKGROUND

## model_bert
print("### Predictions of fine-tuned 'bert-base-cased':\n")
predictions, raw_outputs = model_bert.predict(sample_texts)
# print("Predictions:", predictions)
# print("Raw Outputs:", raw_outputs)
# Decode label predictions
decoded_predictions = label_encoder.inverse_transform(predictions)
print("Decoded Predictions:", decoded_predictions)

## model_biobert
print("\n### Predictions of fine-tuned 'biobert-base-cased-v1.2':\n")
predictions, raw_outputs = model_biobert.predict(sample_texts)
# print("Predictions:", predictions)
# print("Raw Outputs:", raw_outputs)
# Decode label predictions
decoded_predictions = label_encoder.inverse_transform(predictions)
print("Decoded Predictions:", decoded_predictions)


### Predictions of fine-tuned 'bert-base-cased':



0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Decoded Predictions: ['BACKGROUND']

### Predictions of fine-tuned 'biobert-base-cased-v1.2':



/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:2188: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Decoded Predictions: ['BACKGROUND']


/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:2188: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


In [ ]:
#Another Example

sample_texts = ["  A total of @ patients were randomized ."] #correct lable is:RESULTS

## model_bert
print("### Predictions of fine-tuned 'bert-base-cased':\n")
predictions, raw_outputs = model_bert.predict(sample_texts)
# print("Predictions:", predictions)
# print("Raw Outputs:", raw_outputs)
# Decode label predictions
decoded_predictions = label_encoder.inverse_transform(predictions)
print("Decoded Predictions:", decoded_predictions)

## model_biobert
print("\n### Predictions of fine-tuned 'biobert-base-cased-v1.2':\n")
predictions, raw_outputs = model_biobert.predict(sample_texts)
# print("Predictions:", predictions)
# print("Raw Outputs:", raw_outputs)
# Decode label predictions
decoded_predictions = label_encoder.inverse_transform(predictions)
print("Decoded Predictions:", decoded_predictions)

### Predictions of fine-tuned 'bert-base-cased':



0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Decoded Predictions: ['RESULTS']

### Predictions of fine-tuned 'biobert-base-cased-v1.2':



/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:2188: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Decoded Predictions: ['RESULTS']


/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:2188: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


# Data: BC5CDR (BioCreative V CDR corpus)

•	Task: Named Entity Recognition (NER)

•	Paper: Jiao Li et al. in BioCreative V CDR task corpus: a resource for chemical disease relation extraction. Database, 2016.

•	Details: BC5CDR corpus consists of 1500 PubMed articles with 4409 annotated chemicals, 5818 diseases and 3116 chemical-disease interactions.

•	Data source : https://github.com/cambridgeltl/MTL-Bioinformatics-2016/raw/master/data/BC5CDR-IOB/


•	Labels: Chemical, Disease.





**Section 1: Setting up the GPU Environment**

In [ ]:
# Install necessary libraries
!pip install simpletransformers transformers pandas --quiet

import os
import pandas as pd
import requests
import logging
from simpletransformers.ner import NERModel, NERArgs
import torch

# Setup logging
logging.basicConfig(level=logging.DEBUG)
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.WARNING)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.5 MB/s eta 0:00:00
   ━━━

**Section 2: Getting Data**

In [ ]:
# Define the URLs and download the data
Base_URL = "https://github.com/cambridgeltl/MTL-Bioinformatics-2016/raw/master/data/BC5CDR-IOB/"
train_file = "train.tsv"
dev_file = "devel.tsv"
test_file = "test.tsv"

def download_file(url, filename):
    response = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(response.content)
    print(f"{filename} downloaded.")

# Download files
download_file(Base_URL + train_file, "train.tsv")
download_file(Base_URL + dev_file, "devel.tsv")
download_file(Base_URL + test_file, "test.tsv")

train.tsv downloaded.
devel.tsv downloaded.
test.tsv downloaded.


In [ ]:
# Data preprocessing

# Load the data into pandas DataFrames
def load_ner_data(file_path):
    """Load NER data from a file into a pandas DataFrame."""
    sentences = []
    sentence = []
    with open(file_path, 'r') as f:
        for line in f:
            if line.strip() == "":
                if sentence:
                    sentences.append(sentence)
                    sentence = []
            else:
                word, label = line.strip().split("\t")
                sentence.append((word, label))
    # Flatten into a DataFrame
    data = {"sentence_id": [], "words": [], "labels": []}
    for i, sentence in enumerate(sentences):
        for word, label in sentence:
            data["sentence_id"].append(i)
            data["words"].append(word)
            data["labels"].append(label)
    return pd.DataFrame(data)


# Load datasets
train_df = load_ner_data("train.tsv")
dev_df = load_ner_data("devel.tsv")
test_df = load_ner_data("test.tsv")
print("Example of preprocessd_train_data:\n",train_df)


Example of preprocessd_train_data:
         sentence_id        words      labels
0                 0   Selegiline  B-Chemical
1                 0            -           O
2                 0      induced           O
3                 0     postural   B-Disease
4                 0  hypotension   I-Disease
...             ...          ...         ...
118165         4559    synthesis           O
118166         4559           in           O
118167         4559          the           O
118168         4559  vasculature           O
118169         4559            .           O

[118170 rows x 3 columns]


**Section 3: Training and Testing the Model**

In [ ]:
# Optional model configuration
model_args = NERArgs(
    num_train_epochs=1,             # Number of epochs
    train_batch_size=16,            # Batch size for training
    eval_batch_size=16,             # Batch size for evaluation
    overwrite_output_dir=True,      # Overwrite the output directory
    output_dir="./NER_Model_bert",  # Output directory
)

# Define labels for the NER model
custom_labels = train_df["labels"].unique().tolist()
print(f"Custom Labels: {custom_labels}")

Custom Labels: ['B-Chemical', 'O', 'B-Disease', 'I-Disease', 'I-Chemical']


In [ ]:
# Create a NERModel: Pre-trained BERT model="bert-base-cased"
ner_model_bert = NERModel(
    "bert",                         # Use the BERT architecture
    "bert-base-cased",            # Pre-trained BERT model
    labels=custom_labels,  # labels for the NER model
    use_cuda=torch.cuda.is_available(),  # True: Use GPU if available
    args=model_args                 # Model arguments
)


# Train the model
ner_model_bert.train_model(train_df, eval_data=dev_df)

# Evaluate the model
result, model_outputs, preds_list = ner_model_bert.eval_model(test_df)

# Display evaluation results
print("Evaluation Results on fine-tuned 'bert-base-cased':", result)



Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 1 of 1:   0%|          | 0/285 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/300 [00:00<?, ?it/s]

Evaluation Results on fine-tuned 'bert-base-cased': {'eval_loss': 0.10103442649667461, 'precision': 0.7882684185828249, 'recall': 0.8568659457253621, 'f1_score': 0.8211370191132619}


In [ ]:
# Create a NERModel: Pre-trained BERT model="biobert-base-cased-v1.2"
ner_model_biobert = NERModel(
    "bert",                         # Use the BERT architecture
    "dmis-lab/biobert-base-cased-v1.2",            # Pre-trained BERT model
    labels=custom_labels,  # labels for the NER model
    use_cuda=torch.cuda.is_available(),  # True: Use GPU if available
    args=model_args                 # Model arguments
)

# Train the model
ner_model_biobert.train_model(train_df, eval_data=dev_df)

# Evaluate the model
result, model_outputs, preds_list = ner_model_biobert.eval_model(test_df)

# Display evaluation results
print("Evaluation Results on fine-tuned 'biobert-base-cased-v1.2':", result)



/home/AD/zeifa/miniconda3/envs/myenv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 1 of 1:   0%|          | 0/285 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/300 [00:00<?, ?it/s]

Evaluation Results on fine-tuned 'biobert-base-cased-v1.2': {'eval_loss': 0.07844814408260087, 'precision': 0.8397546482652866, 'recall': 0.8938992042440318, 'f1_score': 0.8659814192528167}


**Section 4: Using the Model**

In [ ]:
samples = ["Torsade de pointes ventricular tachycardia during low dose intermittent dobutamine treatment in a patient with dilated cardiomyopathy and congestive heart failure."]
##Disease Entities: Torsade de pointes ventricular tachycardia, Dilated cardiomyopathy, Congestive heart failure
##Chemical Entities: Dobutamine

print("### Predictions of fine-tuned 'bert-base-cased':\n")
predictions, _ = ner_model_bert.predict(samples)
for idx, sample in enumerate(samples):
  print('{}: '.format(idx))
  for word in predictions[idx]:
    print('{}'.format(word))

print("\n### Predictions of fine-tuned 'biobert-base-cased-v1.2':\n")
predictions, _ = ner_model_biobert.predict(samples)
for idx, sample in enumerate(samples):
  print('{}: '.format(idx))
  for word in predictions[idx]:
    print('{}'.format(word))

### Predictions of fine-tuned 'bert-base-cased':



  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

0: 
{'Torsade': 'B-Chemical'}
{'de': 'O'}
{'pointes': 'O'}
{'ventricular': 'B-Disease'}
{'tachycardia': 'I-Disease'}
{'during': 'O'}
{'low': 'O'}
{'dose': 'O'}
{'intermittent': 'O'}
{'dobutamine': 'B-Chemical'}
{'treatment': 'O'}
{'in': 'O'}
{'a': 'O'}
{'patient': 'O'}
{'with': 'O'}
{'dilated': 'B-Disease'}
{'cardiomyopathy': 'I-Disease'}
{'and': 'O'}
{'congestive': 'B-Disease'}
{'heart': 'I-Disease'}
{'failure.': 'I-Disease'}

### Predictions of fine-tuned 'biobert-base-cased-v1.2':



  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

0: 
{'Torsade': 'B-Disease'}
{'de': 'I-Disease'}
{'pointes': 'I-Disease'}
{'ventricular': 'B-Disease'}
{'tachycardia': 'I-Disease'}
{'during': 'O'}
{'low': 'O'}
{'dose': 'O'}
{'intermittent': 'O'}
{'dobutamine': 'B-Chemical'}
{'treatment': 'O'}
{'in': 'O'}
{'a': 'O'}
{'patient': 'O'}
{'with': 'O'}
{'dilated': 'B-Disease'}
{'cardiomyopathy': 'I-Disease'}
{'and': 'O'}
{'congestive': 'B-Disease'}
{'heart': 'I-Disease'}
{'failure.': 'I-Disease'}


In [ ]:
# Another Example

samples = ["Meloxicam induced liver toxicity."]
##Disease Entities: Liver toxicity
##Chemical Entities: Meloxicam

print("### Predictions of fine-tuned 'bert-base-cased':\n")
predictions, _ = ner_model_bert.predict(samples)
for idx, sample in enumerate(samples):
  print('{}: '.format(idx))
  for word in predictions[idx]:
    print('{}'.format(word))

print("\n### Predictions of fine-tuned 'biobert-base-cased-v1.2':\n")
predictions, _ = ner_model_biobert.predict(samples)
for idx, sample in enumerate(samples):
  print('{}: '.format(idx))
  for word in predictions[idx]:
    print('{}'.format(word))

### Predictions of fine-tuned 'bert-base-cased':



  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

0: 
{'Meloxicam': 'B-Chemical'}
{'induced': 'O'}
{'liver': 'B-Disease'}
{'toxicity.': 'I-Disease'}

### Predictions of fine-tuned 'biobert-base-cased-v1.2':



  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

0: 
{'Meloxicam': 'B-Chemical'}
{'induced': 'O'}
{'liver': 'B-Disease'}
{'toxicity.': 'I-Disease'}


In [ ]:
# Another Example

samples = ["This first case of meloxicam related liver toxicity demonstrates the potential of this drug to induce hepatic damage."]
##Disease Entities: Liver toxicity, Hepatic damage
##Chemical Entities: Meloxicam

print("Predictions of fine-tuned 'bert-base-cased' :")
predictions, _ = ner_model_bert.predict(samples)
for idx, sample in enumerate(samples):
  print('{}: '.format(idx))
  for word in predictions[idx]:
    print('{}'.format(word))

print("Predictions of fine-tuned 'biobert-base-cased-v1.2' :")
predictions, _ = ner_model_biobert.predict(samples)
for idx, sample in enumerate(samples):
  print('{}: '.format(idx))
  for word in predictions[idx]:
    print('{}'.format(word))

Predictions of fine-tuned 'bert-base-cased' :


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

0: 
{'This': 'O'}
{'first': 'O'}
{'case': 'O'}
{'of': 'O'}
{'meloxicam': 'B-Chemical'}
{'related': 'O'}
{'liver': 'B-Disease'}
{'toxicity': 'I-Disease'}
{'demonstrates': 'O'}
{'the': 'O'}
{'potential': 'O'}
{'of': 'O'}
{'this': 'O'}
{'drug': 'O'}
{'to': 'O'}
{'induce': 'O'}
{'hepatic': 'B-Disease'}
{'damage.': 'I-Disease'}
Predictions of fine-tuned 'biobert-base-cased-v1.2' :


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

0: 
{'This': 'O'}
{'first': 'O'}
{'case': 'O'}
{'of': 'O'}
{'meloxicam': 'B-Chemical'}
{'related': 'O'}
{'liver': 'B-Disease'}
{'toxicity': 'I-Disease'}
{'demonstrates': 'O'}
{'the': 'O'}
{'potential': 'O'}
{'of': 'O'}
{'this': 'O'}
{'drug': 'O'}
{'to': 'O'}
{'induce': 'O'}
{'hepatic': 'B-Disease'}
{'damage.': 'I-Disease'}
